In [1]:
import sys
sys.path.insert(0, "submodules/dc-egm/src/")
import numpy as np

In [2]:
from dcegm.solve import get_solve_function

In [5]:
options_test = {
     # mandatory
    "n_periods": 50, # 25 + 50 = 75
    "n_discrete_choices": 3,
    "n_exog_states": 1,
    # custom: model structure
    "start_age": 25,
    "resolution_age": 60,
    # custom: policy environment
    "minimum_SRA": 67, 
    "unemployment_benefits": 5,
    "pension_point_value": 0.3,
    "early_retirement_penalty": 0.036,
    # custom: params estimated outside model
    "belief_update_increment": 0.05,
    "gamma_0": 10,
    "gamma_1": 1,
    "gamma_2": -0.1,
    "interest_rate": 0.03
}

params_dict_test = {
    "mu": 0.5,
    "delta": 4  
}

# State space - No need to be jax compatible

In [63]:
def create_state_space(options):
    n_periods = options["n_periods"]
    n_choices = options["n_discrete_choices"]
    n_exog_states = options["n_exog_states"]
    resolution_age = options["resolution_age"]
    start_age = options["start_age"]
    belief_update_increment = options["belief_update_increment"]
    
    # The highest policy state, we consider belongs to the expectation of the youngest.
    n_policy_states = resolution_age - start_age

    # minimum retirement age is 4 years before the lowest statutory ret age 
    earliest_ret_age = options["minimum_SRA"] - 4
    # maximum (conceivable) retirement age is given by lowest SRA plus the projection of the youngest
    max_ret_age = options["minimum_SRA"] + belief_update_increment * n_policy_states

    # shape = (n_periods, n_choices, n_exog_states)
    state_space = []
    
    shape = (n_periods, n_choices, n_periods, n_policy_states, 1)
    
    map_state_to_index = np.full(shape, fill_value=-9999, dtype=np.int64)
    i = 0

    for period in range(n_periods):
        for lag_choice in range(n_choices):
            # You cannot have more experience than your age
            for exp in range(period + 1):
                # The policy state we need to consider increases by one increment per period.
                for policy_state in range(period + 1):
                    age = start_age + period
                    # You cannot retire before the earliest retirement age
                    if (age <= earliest_ret_age) & (lag_choice == 2):
                        continue
                    # After the maximum retirement age, you must be retired
                    elif (age > max_ret_age) & (lag_choice != 2):
                        continue
                    # Starting from resolution age, there is no more adding of policy states. 
                    elif policy_state > n_policy_states - 1:
                        continue
                    # If you have not worked last period, you can't have worked all your live 
                    elif (lag_choice != 1) & (period == exp) & (period > 0):
                        continue
                    else:
                        state_space += [[period, lag_choice, exp, policy_state, 0]]
                        map_state_to_index[period, lag_choice, exp, policy_state, 0] = i
                        i +=1

    return np.array(state_space), map_state_to_index


def get_choice_set(state, map_state_to_index):

    # if you're younger than min SRA, you cannot retire
    if state[0] < 67 - 25:
        return np.array([0, 1])
     # After the maximum retirement age, you must be retired
    elif state[0] > 69 - 25:
        return np.array([2])
    # retirement is absorbing
    elif state[1] == 2:
        return np.array([2])
    else:
        return np.array([0, 1, 2])
    
def update_state(state, choice):
    state_next = state.copy()
    
    # age increases by one
    state_next[0] += 1
    
    # Set choice as lag choice in next state
    state_next[1] = choice
    
    # experience increases by one if working
    if choice == 1:
        state_next[2] += 1
    
    # expected SRA increases by one increment
    if state[0] + 25 < 60:
        state_next[3] += 1
        
    return state_next
            

# Utility funcs

In [6]:
def utility_func(consumption, choice, params_dict):
    mu = params_dict["mu"]
    delta = params_dict["delta"]
    is_working = choice == 1
    utility = consumption ** (1- mu) / (1 - mu) - delta * is_working
    return utility


def marg_utility(consumption, params_dict):
    mu = params_dict["mu"]
    marg_util = consumption ** -mu
    return marg_util


def inverse_marginal(marginal_utility, params_dict):
    mu = params_dict["mu"]
    return marginal_utility ** (-1/mu)


utility_functions = {
        "utility": utility_func,
        "inverse_marginal_utility": inverse_marginal,
        "marginal_utility": marg_utility,
    }
    

# Budget constraint

In [7]:
def budget_constraint(state_beginning_of_period, # s_t, with d_{t-1} at s_t[1]
                      savings_end_of_previous_period, # A_{t-1}
                      income_shock_previous_period, # epsilon_{t - 1}
                      params,
                      options):
    
    unemployment_benefits = options["unemployment_benefits"]
    
    # fetch necessary parameters (gammas for wage, lambdas & ERP for pension)
    gamma_0 = options["gamma_0"]
    gamma_1 = options["gamma_1"]
    gamma_2 = options["gamma_2"]
    lambd = options["pension_point_value"]
    ERP = options["early_retirement_penalty"]
    
    # read out state
    age = state_beginning_of_period[0]
    lag_choice = state_beginning_of_period[1]
    experience = state_beginning_of_period[2]
    current_SRA = state_beginning_of_period[3]
    actual_retirement_age = 68 # B: das hier fehlt noch im state space
    
    # calculate applicable SRA and pension deduction factor
    resolution_SRA = current_SRA - (age - options["resolution_age"])*options["belief_update_increment"]
    pension_deduction_factor = 1
    if actual_retirement_age < resolution_SRA:
        pension_deduction_factor = 1 - (actual_retirement_age - resolution_SRA)*ERP
    
    # calculate choice-specific income
    income = [] * options["n_discrete_choices"]
    
    income[0] = options["unemployment_benefits"]
    income[1] = gamma_0 + gamma_1*experience + gamma_2*experience**2 + income_shock_previous_period
    income[2] = lambd*experience*pension_deduction_factor 
        
    # calculate beginning of period wealth M_t
    wealth = (1 + options["interest_rate"]) * savings_end_of_previous_period + income[lag_choice] 
    

# Dumm exog process

In [70]:
def dummy_exog(state, params_dict):
    return np.array([1])

In [64]:
state_space, map_state_to_index = create_state_space(options_test)
period_to_inspect = 0

state_space[state_space[:, 0] == period_to_inspect]

array([[0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0]])

In [71]:
get_solve_function?